In [1]:
# Core Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from scipy import stats
from scipy.stats import randint, uniform

## IMPORT & EXPLORE

In [2]:
import os

In [35]:
base_dir = r"C:\Users\User\Desktop\DATA SCIENCE\DataSets\flip_book_data"

# Dictionary to store each DataFrame with its dynamic name
dataframes = {}

# Iterate through subdirectories in base_dir
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        csv_file = f"flip_books_{folder_name}.csv"
        csv_path = os.path.join(folder_path, csv_file)
        if os.path.exists(csv_path):
            var_name = f"flip_books_{folder_name}"
            dataframes[var_name] = pd.read_csv(csv_path)

# Optional: assign variables to globals if needed
globals().update(dataframes)

In [4]:
flip_books_art.sample(3)

,title,price_current,price_original,discount,publisher,language,binding,publication_date,isbn,pages,...,thickness,product_code,availability,rating,reviews_count,description,main_image_url,additional_images,local_image_path,book_url
324,Волшебные цветы. Нарисуй акварелью,4674 ₸,4674 ₸,-20%,«Контэнт» специализируется на издании литерату...,NaN,мягкая обложка,NaN,NaN,NaN,...,NaN,NaN,10 июня,NaN,0,Уникальный проект для художников-акварелистов!...,//s.f.kz/prod/4270/4269656_1000.jpg,//s.f.kz/prod/photo/8074/8073218_240.jpg; //s....,/root/flip_book/data/art/images/Волшебные_цвет...,https://www.flip.kz/catalog?prod=4269656
851,Культура и эволюция. Теоретические исследования,5559 ₸,NaN,NaN,"""Евразия"" существует с 1996 года. Как свидетел...",NaN,твердый переплет,NaN,NaN,NaN,...,NaN,NaN,11 июня,NaN,0,Новая книга знаменитого санкт-петербургского у...,//s.f.kz/prod/1204/1203602_1000.jpg,NaN,/root/flip_book/data/art/images/Культура_и_эво...,https://www.flip.kz/catalog?prod=1203602
1265,Трансформации драмы в истории: Структуры поряд...,7741 ₸,NaN,NaN,«Новое литературное обозрение» трудится на нив...,NaN,твердый переплет,NaN,NaN,NaN,...,NaN,NaN,11 июня,NaN,0,"Как и все художественные жанры, драма на протя...",//s.f.kz/prod/3854/3853642_1000.jpg,//s.f.kz/prod/photo/7229/7228694_240.jpg; //s....,/root/flip_book/data/art/images/Трансформации_...,https://www.flip.kz/catalog?prod=3853642


In [5]:
flip_books_art.columns

Index(['title', 'price_current', 'price_original', 'discount', 'publisher',
       'language', 'binding', 'publication_date', 'isbn', 'pages', 'height',
       'width', 'thickness', 'product_code', 'availability', 'rating',
       'reviews_count', 'description', 'main_image_url', 'additional_images',
       'local_image_path', 'book_url'],
      dtype='object')

## CLEAN & PREPARE

#### FILTERING

In [36]:

filtered_columns = [ "title", "price_original", "discount", "publisher",
                     "binding", "reviews_count", "description",
                     "book_url", "local_image_path", "main_image_url"]


In [37]:
# Regex binding filter
binding_filter = lambda df: df[
    df['binding'].str.lower().isin([
        'твердая обложка',
        'твердый переплет',
        'мягкая обложка',
        'мягкий переплет'
    ])
]
# Apply filter, select columns, and add "category" column
flip_books_art = binding_filter(flip_books_art)[filtered_columns].assign(category='art')
flip_books_kids = binding_filter(flip_books_kids)[filtered_columns].assign(category='kids')
flip_books_history = binding_filter(flip_books_history)[filtered_columns].assign(category='history')
flip_books_biography = binding_filter(flip_books_biography)[filtered_columns].assign(category='biography')
flip_books_education = binding_filter(flip_books_education)[filtered_columns].assign(category='education')
flip_books_programming = binding_filter(flip_books_programming)[filtered_columns].assign(category='programming')
flip_books_romance = binding_filter(flip_books_romance)[filtered_columns].assign(category='romance')
flip_books_psychology = binding_filter(flip_books_psychology)[filtered_columns].assign(category='psychology')
flip_books_science = binding_filter(flip_books_science)[filtered_columns].assign(category='science')
flip_books_fantasy = binding_filter(flip_books_fantasy)[filtered_columns].assign(category='fantasy')


#### COMBINE

In [38]:
flip_books_data = pd.concat([
                            flip_books_art,
                            flip_books_kids,
                            flip_books_history,
                            flip_books_biography,
                            flip_books_education,
                            flip_books_programming,
                            flip_books_romance,
                            flip_books_psychology,
                            flip_books_science,
                            flip_books_fantasy
                        ], ignore_index=True)

#### NaNs

In [39]:
flip_books_data['discount'] = (
    flip_books_data['discount']
    .fillna('0')                          # Replace NaN with '0'
    .astype(str)                          # Ensure all are strings
    .str.replace('%', '', regex=False)   # Remove % sign
    .astype(float)                        # Convert to float first
    .astype(int)                          # Then to int
)

flip_books_data['discount'] = flip_books_data['discount'].abs()


In [40]:
flip_books_data.isna().sum()

title                  0
price_original      6150
discount               0
publisher           2445
binding                0
reviews_count          0
description            0
book_url               0
local_image_path       1
main_image_url         0
category               0
dtype: int64

In [41]:
flip_books_data['publisher'] = flip_books_data['publisher'].fillna( "unknown" )

In [42]:
flip_books_data.isna().sum()

title                  0
price_original      6150
discount               0
publisher              0
binding                0
reviews_count          0
description            0
book_url               0
local_image_path       1
main_image_url         0
category               0
dtype: int64

In [43]:
flip_books_data = flip_books_data.dropna()

flip_books_data.shape


(5525, 11)

In [44]:
flip_books_data.isna().sum().any()

False

In [45]:
flip_books_data['price_original'] = (
    flip_books_data['price_original']  # replace with your actual column name
    .astype(str)
    .str.extract(r'(\d[\d\s]*)')  # extract number part
    .fillna('0')
    [0].str.replace(r'\s+', '', regex=True)  # remove spaces
    .astype(int)
)


In [46]:
flip_books_data['windows_image_path'] = flip_books_data['local_image_path'].apply(
    lambda x: r'C:\Users\User\Desktop\DATA SCIENCE\DataSets\flip_book_data' + x.split('/flip_book/data')[1].replace('/', '\\')
)


In [47]:
flip_books_data.sample(5)

,title,price_original,discount,publisher,binding,reviews_count,description,book_url,local_image_path,main_image_url,category,windows_image_path
6596,По осколкам твоего сердца. Твоё сердце будет р...,6029,20,Clever — это ведущий издатель в мире образован...,твердый переплет,86,Твое сердце будет разбито\n\r\nНовый роман Анн...,https://www.flip.kz/catalog?prod=3166818,/root/flip_book/data/romance/images/По_осколка...,//s.f.kz/prod/3167/3166818_1000.jpg,romance,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...
9427,"Пространство, время и движение. Величайшие иде...",8565,35,предлагает образовательную литературу для студ...,твердый переплет,0,"Ученые знают о том, как устроен наш мир, Вселе...",https://www.flip.kz/catalog?prod=4267367,/root/flip_book/data/science/images/Пространст...,//s.f.kz/prod/4268/4267367_1000.jpg,science,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...
3641,Тейлор Свифт. Дневник свифти,5679,25,«Эксмо» — крупнейшее издательство в России,твердый переплет,1,Какого цвета твои мечты о будущем? Какого цвет...,https://www.flip.kz/catalog?prod=4165636,/root/flip_book/data/biography/images/Тейлор_С...,//s.f.kz/prod/4166/4165636_1000.jpg,biography,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...
6798,Король Людовик Святой,3056,50,«Вече» — ваш надежный проводник в мире литерат...,твердый переплет,0,"1250 год. Седьмой крестовый поход, возглавляем...",https://www.flip.kz/catalog?prod=3780722,/root/flip_book/data/romance/images/Король_Люд...,//s.f.kz/prod/3781/3780722_1000.jpg,romance,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...
10028,Страшные русские сказки,4234,25,«АСТ» — это ведущий издательский дом в России,твердый переплет,0,"Сказки, которые мы любим с детства, не всегда ...",https://www.flip.kz/catalog?prod=4340262,/root/flip_book/data/science/images/Страшные_р...,//s.f.kz/prod/4341/4340262_1000.jpg,science,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...


#### ENCODING

In [48]:
for column in ['publisher', 'binding']:

    flip_books_data[f'{column}_code'] = flip_books_data[column].astype('category').cat.codes

    

In [71]:
category_mapping = {
    'fantasy': 134,
    'romance': 142,
    'biography': 158,
    'programming' : 37,
    'science': 386,
    'art': 53,
    'education': 279,
    'history': 5863,
    'kids': 43,
    'psychology': 505
}

flip_books_data['category_id'] = flip_books_data['category'].map(category_mapping)


In [72]:
flip_books_data.sample(5)

,title,price_original,discount,publisher,binding,reviews_count,description,book_url,local_image_path,main_image_url,category,windows_image_path,publisher_code,binding_code,category_id
2451,Математическая смекалка. Лучшие логические зад...,6893,20,«АСТ» — это ведущий издательский дом в России,твердый переплет,13,Книга мэтра отечественной научно-популярной ли...,https://www.flip.kz/catalog?prod=1231055,/root/flip_book/data/kids/images/Математическа...,//s.f.kz/prod/1232/1231055_1000.jpg,kids,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...,35,1,43
2234,Адам миы. Дәрігер Сантьяго Рамон-и-Кахальдің т...,4196,25,«Фолиант» было основано в 1996 году и с тех по...,твердый переплет,0,Бұл кітапта Сантьяго есімді дәрігер мидың нелі...,https://www.flip.kz/catalog?prod=3216416,/root/flip_book/data/kids/images/Адам_миы_Дәрі...,//s.f.kz/prod/3217/3216416_1000.jpg,kids,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...,124,1,43
6756,Қаһар. Көшпенділер,3717,25,«Фолиант» было основано в 1996 году и с тех по...,твердый переплет,0,Көшпенділер — Ілияс Есенберлиннің «Алмас қылыш...,https://www.flip.kz/catalog?prod=4306675,/root/flip_book/data/romance/images/Қаһар_Көшп...,//s.f.kz/prod/4307/4306675_1000.jpg,romance,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...,124,1,142
5744,"УМка Годовой курс занятий, 5-6 лет",2360,50,unknown,мягкая обложка,0,чебное пособие «Т. А. Ткаченко. Годовой курс з...,https://www.flip.kz/catalog?prod=2315130,/root/flip_book/data/education/images/УМка_Год...,//s.f.kz/prod/2316/2315130_1000.jpg,education,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...,30,0,279
2001,Азбукварик Два веселых гуся. Нажми на кнопку и...,1482,20,unknown,твердый переплет,7,Хорошая музыкальная книжка. звук не тихий и не...,https://www.flip.kz/catalog?prod=402669,/root/flip_book/data/kids/images/Азбукварик_Дв...,//s.f.kz/prod/403/402669_1000.jpg,kids,C:\Users\User\Desktop\DATA SCIENCE\DataSets\fl...,30,1,43


## SAVE

In [74]:
flip_books_data.to_csv('flip_books_data_cleaned.csv.gz', compression = 'gzip',
                       index = False)